In [10]:
import pandas as pd
import numpy as np
import json
from collections import defaultdict
import pickle

In [11]:
# 讀取數據
account_info = pd.read_csv('.\Train\\train_account_info.csv')
customer_info = pd.read_csv('.\Train\\train_customer_info.csv')
account_transactions = pd.read_csv('.\Train\\train_account_transactions.csv')
suspicious_accounts = pd.read_csv('.\Train\\train_suspicious_accounts.csv')

# 測試集數據
test_account_info = pd.read_csv('.\Test\\test_account_info.csv')
test_customer_info = pd.read_csv('.\Test\\test_customer_info.csv')
test_transactions = pd.read_csv('.\Test\\test_account_transactions.csv')

In [12]:
account_customer_map = account_info[['account_number', 'customer_id']].drop_duplicates()

# 合併客戶信息
merged_base = pd.merge(
    account_customer_map,
    customer_info,
    on='customer_id',
    how='left'
)

# 合併帳戶信息
merged_base = pd.merge(
    merged_base,
    account_info.drop('customer_id', axis=1),
    on='account_number',
    how='left'
)

# 添加可疑帳戶標記
suspicious_accounts_set = set(suspicious_accounts['account_number'].unique())
merged_base['is_suspicious'] = merged_base['account_number'].apply(
    lambda x: 1 if x in suspicious_accounts_set else 0
)

print(f"基礎合併數據形狀: {merged_base.shape}")
print(merged_base.head())

基礎合併數據形狀: (24969, 10)
  account_number customer_id  aum_amt  age  income_level  region_code  \
0       ACCT6068      ID5684   256930   61          25.0         12.0   
1      ACCT11459     ID10838       65   57           NaN         12.0   
2      ACCT15832     ID15012    14438   56         126.0         12.0   
3      ACCT15612     ID14797    43872   46           NaN         12.0   
4      ACCT18659     ID17677  2578166   72          25.0         12.0   

   is_unreachable  is_digital_account  account_open_date  is_suspicious  
0               0                   0               8400              0  
1               0                   0               9569              0  
2               0                   0               8380              0  
3               0                   0              10610              0  
4               0                   0               9666              0  


In [13]:
# 初始化存儲結構
account_data = {}

# 首先填充帳戶和客戶信息
for _, row in merged_base.iterrows():
    account_number = row['account_number']
    
    # 創建帳戶數據字典
    account_data[account_number] = {
        'account_info': row.to_dict(),  # 包含所有帳戶和客戶信息
        'transactions': []  # 初始化空交易列表
    }

In [14]:
# 統計需要處理的交易總數
total_transactions = len(account_transactions)
print(f"開始處理 {total_transactions} 筆交易...")

# 批次處理交易數據以避免內存問題
batch_size = 50000
num_batches = (total_transactions + batch_size - 1) // batch_size

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, total_transactions)
    
    print(f"處理交易批次 {i+1}/{num_batches} (行 {start_idx} 到 {end_idx})...")
    
    # 獲取當前批次的交易
    batch_transactions = account_transactions.iloc[start_idx:end_idx]
    
    # 遍歷此批次的交易並添加到相應的帳戶
    for _, txn in batch_transactions.iterrows():
        account_number = txn['account_number']
        
        # 確保帳戶存在於我們的數據結構中
        if account_number in account_data:
            # 將交易轉換為字典並添加到帳戶的交易列表
            account_data[account_number]['transactions'].append(txn.to_dict())

# 4. 分析數據結構和示例
print("\n分析字典式數據結構...")

# 計算總帳戶數
total_accounts = len(account_data)
print(f"總帳戶數: {total_accounts}")

# 計算有交易記錄的帳戶數
accounts_with_transactions = sum(1 for acc in account_data.values() if len(acc['transactions']) > 0)
print(f"有交易記錄的帳戶數: {accounts_with_transactions}")
print(f"無交易記錄的帳戶數: {total_accounts - accounts_with_transactions}")

# 計算每個帳戶的平均交易數
total_txns = sum(len(acc['transactions']) for acc in account_data.values())
avg_txns_per_account = total_txns / accounts_with_transactions if accounts_with_transactions > 0 else 0
print(f"每個帳戶的平均交易數: {avg_txns_per_account:.2f}")

# 找到交易數量最多的帳戶
max_txns_account = max(account_data.items(), key=lambda x: len(x[1]['transactions']), default=(None, {'transactions': []}))
if max_txns_account[0] is not None:
    print(f"交易數量最多的帳戶: {max_txns_account[0]} 有 {len(max_txns_account[1]['transactions'])} 筆交易")

# 5. 示例：顯示一些帳戶的數據
print("\n示例帳戶數據:")
# 獲取一些帳戶號碼作為示例
sample_accounts = list(account_data.keys())[:5]

for i, account in enumerate(sample_accounts):
    acc_data = account_data[account]
    print(f"\n帳戶 {i+1}: {account}")
    print(f"  客戶ID: {acc_data['account_info']['customer_id']}")
    print(f"  帳戶開立日期: {acc_data['account_info']['account_open_date']}")
    print(f"  是否為可疑帳戶: {'是' if acc_data['account_info']['is_suspicious'] == 1 else '否'}")
    
    # 顯示交易摘要
    txns = acc_data['transactions']
    print(f"  交易數量: {len(txns)}")
    
    # 如果有交易，顯示前3筆
    if txns:
        print("  交易示例:")
        for j, txn in enumerate(txns[:3]):
            print(f"    交易 {j+1}: 日期={txn.get('transaction_date')}, 金額={txn.get('transaction_amount')}, 方向={'入帳' if txn.get('transaction_direction') == 1 else '出帳'}")


開始處理 206333 筆交易...
處理交易批次 1/5 (行 0 到 50000)...
處理交易批次 2/5 (行 50000 到 100000)...
處理交易批次 3/5 (行 100000 到 150000)...
處理交易批次 4/5 (行 150000 到 200000)...
處理交易批次 5/5 (行 200000 到 206333)...

分析字典式數據結構...
總帳戶數: 24969
有交易記錄的帳戶數: 24969
無交易記錄的帳戶數: 0
每個帳戶的平均交易數: 8.26
交易數量最多的帳戶: ACCT28178 有 508 筆交易

示例帳戶數據:

帳戶 1: ACCT6068
  客戶ID: ID5684
  帳戶開立日期: 8400
  是否為可疑帳戶: 否
  交易數量: 2
  交易示例:
    交易 1: 日期=18264, 金額=8531, 方向=出帳
    交易 2: 日期=18278, 金額=752, 方向=出帳

帳戶 2: ACCT11459
  客戶ID: ID10838
  帳戶開立日期: 9569
  是否為可疑帳戶: 否
  交易數量: 12
  交易示例:
    交易 1: 日期=18260, 金額=1554, 方向=入帳
    交易 2: 日期=18260, 金額=201, 方向=出帳
    交易 3: 日期=18260, 金額=605, 方向=出帳

帳戶 3: ACCT15832
  客戶ID: ID15012
  帳戶開立日期: 8380
  是否為可疑帳戶: 否
  交易數量: 6
  交易示例:
    交易 1: 日期=18263, 金額=13995, 方向=出帳
    交易 2: 日期=18270, 金額=10870, 方向=入帳
    交易 3: 日期=18277, 金額=34995, 方向=出帳

帳戶 4: ACCT15612
  客戶ID: ID14797
  帳戶開立日期: 10610
  是否為可疑帳戶: 否
  交易數量: 2
  交易示例:
    交易 1: 日期=18268, 金額=2018, 方向=出帳
    交易 2: 日期=18278, 金額=114, 方向=出帳

帳戶 5: ACCT18659
  客戶ID: ID17677
  帳戶開立日

In [15]:
# 6. 保存數據結構
print("\n保存數據結構...")

# 方法1: 使用pickle保存整個數據結構（適合Python內部使用）
with open('account_data_structure.pkl', 'wb') as f:
    pickle.dump(account_data, f)
print("數據結構已保存為pickle文件: account_data_structure.pkl")


保存數據結構...
數據結構已保存為pickle文件: account_data_structure.pkl


In [ ]:
# 方法2: 將結構轉換為JSON格式（更通用，但可能會很大）
# 注意: 這可能會產生非常大的文件，取決於數據量
try:
    with open('account_data_sample.json', 'w') as f:
        # 只保存前100個帳戶到JSON以避免文件過大
        sample_data = {k: account_data[k] for k in list(account_data.keys())}
        json.dump(sample_data, f, indent=2)
except Exception as e:
    print(f"保存JSON時出錯: {e}")

# 7. 示例：如何使用這個數據結構進行分析
print("\n示例分析:")

# 示例1: 計算每個帳戶的交易統計資訊
print("示例1: 計算交易統計資訊")
account_stats = {}

for account, data in account_data.items():
    txns = data['transactions']
    if not txns:
        continue
    
    # 交易金額統計
    amounts = [t.get('transaction_amount', 0) for t in txns]
    
    # 入帳和出帳交易
    incoming = [t for t in txns if t.get('transaction_direction') == 1]
    outgoing = [t for t in txns if t.get('transaction_direction') == 0]
    
    # 收集統計資訊
    account_stats[account] = {
        'txn_count': len(txns),
        'avg_amount': np.mean(amounts) if amounts else 0,
        'max_amount': max(amounts) if amounts else 0,
        'incoming_count': len(incoming),
        'outgoing_count': len(outgoing),
        'incoming_ratio': len(incoming) / len(txns) if txns else 0,
        'is_suspicious': data['account_info'].get('is_suspicious', 0)
    }

# 將統計資訊轉換為DataFrame以便分析
stats_df = pd.DataFrame.from_dict(account_stats, orient='index')
print("交易統計資訊數據框頭部:")
print(stats_df.head())

# 比較可疑帳戶和非可疑帳戶的統計差異
suspicious_stats = stats_df[stats_df['is_suspicious'] == 1].mean()
non_suspicious_stats = stats_df[stats_df['is_suspicious'] == 0].mean()

print("\n可疑帳戶 vs 非可疑帳戶平均統計:")
comparison = pd.DataFrame({
    '可疑帳戶': suspicious_stats,
    '非可疑帳戶': non_suspicious_stats,
    '差異比例': suspicious_stats / non_suspicious_stats
})
print(comparison)

print("\n數據結構創建和分析完成!")

前100個帳戶的樣本數據已保存為JSON文件: account_data_sample.json

示例分析:
示例1: 計算交易統計資訊
交易統計資訊數據框頭部:
           txn_count    avg_amount  max_amount  incoming_count  \
ACCT6068           2   4641.500000        8531               0   
ACCT11459         12   1402.916667        5018               7   
ACCT15832          6  19318.166667       34995               3   
ACCT15612          2   1066.000000        2018               0   
ACCT18659          9  71438.555556       99995               1   

           outgoing_count  incoming_ratio  is_suspicious  
ACCT6068                0        0.000000              0  
ACCT11459               0        0.583333              0  
ACCT15832               0        0.500000              0  
ACCT15612               0        0.000000              0  
ACCT18659               0        0.111111              0  

可疑帳戶 vs 非可疑帳戶平均統計:
                         可疑帳戶         非可疑帳戶      差異比例
txn_count           24.932500      7.992185  3.119610
avg_amount       44127.763504  16815.3

In [17]:
import json
import os
from datetime import datetime

# 1. 讀取現有的JSON文件
print("讀取現有的JSON文件...")
try:
    with open('account_data_sample.json', 'r') as f:
        account_data = json.load(f)
    print(f"成功讀取JSON文件，共有 {len(account_data)} 個帳戶")
except FileNotFoundError:
    print("找不到JSON文件: account_data_sample.json")
    exit(1)
except json.JSONDecodeError:
    print("JSON解析錯誤，請確認文件格式正確")
    exit(1)

# 2. 分離可疑帳戶和非可疑帳戶
print("\n分離可疑帳戶和非可疑帳戶...")

suspicious_accounts = {}
non_suspicious_accounts = {}

for account_number, data in account_data.items():
    # 檢查帳戶是否被標記為可疑
    # 根據您的數據結構，可能需要調整此處的路徑
    if data['account_info'].get('is_suspicious', 0) == 1:
        suspicious_accounts[account_number] = data
    else:
        non_suspicious_accounts[account_number] = data

print(f"可疑帳戶數量: {len(suspicious_accounts)}")
print(f"非可疑帳戶數量: {len(non_suspicious_accounts)}")

# 3. 創建輸出目錄（如果不存在）
output_dir = "split_accounts"
os.makedirs(output_dir, exist_ok=True)

# 4. 生成時間戳（用於文件名）
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# 5. 保存可疑帳戶JSON
suspicious_file = os.path.join(output_dir, f"suspicious_accounts.json")
with open(suspicious_file, 'w', encoding='utf-8') as f:
    json.dump(suspicious_accounts, f, indent=2, ensure_ascii=False)
print(f"可疑帳戶已保存為: {suspicious_file}")

# 6. 保存非可疑帳戶JSON
non_suspicious_file = os.path.join(output_dir, f"non_suspicious_accounts.json")
with open(non_suspicious_file, 'w', encoding='utf-8') as f:
    json.dump(non_suspicious_accounts, f, indent=2, ensure_ascii=False)
print(f"非可疑帳戶已保存為: {non_suspicious_file}")

# 7. 輸出兩種帳戶的基本統計比較
print("\n帳戶統計比較：")

# 計算交易數量
suspicious_txn_count = sum(len(data['transactions']) for data in suspicious_accounts.values())
non_suspicious_txn_count = sum(len(data['transactions']) for data in non_suspicious_accounts.values())

# 計算平均每個帳戶的交易數
suspicious_avg_txn = suspicious_txn_count / len(suspicious_accounts) if suspicious_accounts else 0
non_suspicious_avg_txn = non_suspicious_txn_count / len(non_suspicious_accounts) if non_suspicious_accounts else 0

print(f"可疑帳戶總交易數: {suspicious_txn_count}")
print(f"非可疑帳戶總交易數: {non_suspicious_txn_count}")
print(f"可疑帳戶平均交易數: {suspicious_avg_txn:.2f}")
print(f"非可疑帳戶平均交易數: {non_suspicious_avg_txn:.2f}")

print("\n分離完成!")

讀取現有的JSON文件...
成功讀取JSON文件，共有 24969 個帳戶

分離可疑帳戶和非可疑帳戶...
可疑帳戶數量: 400
非可疑帳戶數量: 24569
可疑帳戶已保存為: split_accounts\suspicious_accounts.json
非可疑帳戶已保存為: split_accounts\non_suspicious_accounts.json

帳戶統計比較：
可疑帳戶總交易數: 9973
非可疑帳戶總交易數: 196360
可疑帳戶平均交易數: 24.93
非可疑帳戶平均交易數: 7.99

分離完成!
